In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import fetch_data
reload(fetch_data)

Read in just the testing and training data.

In [ ]:
xgbInput = fetch_data.clfInput()
#xgbInput.sessions_ftrEng()
#xgbInput.users_ftrEng()
#xgbInput.one_hot()
#xgbInput.binarize_targets()
#xgbInput.split_data()

In [ ]:
param = {'num_class': 12, 'objective': 'multi:softprob', 'seed': 0}
param['eta'] = 0.20
param['max_depth'] = 6
param['subsample'] = .5
param['col_sample_bytree'] = .6
results = {}
cv_train = pd.read_pickle('cv_results/actions_e20/train_err.p')
cv_valid = pd.read_pickle('cv_results/actions_e20/validate_err.p')
nrounds = 40
for train_indx, valid_indx in cv_bydate(xgbInput):
    dtrain = xgb.DMatrix(xgbInput.train_X[train_indx], label = xgbInput.train_Y[train_indx],
                missing = -1)
    dvalid = xgb.DMatrix(xgbInput.train_X[valid_indx], label = xgbInput.train_Y[valid_indx],
                missing = -1)
    evallist = [(dtrain, 'train'), (dvalid, 'eval')]
    bst = xgb.train(param, dtrain, nrounds, evallist, feval = calc_ndcg.evalerror, evals_result = results)
    cv_train = pd.concat([cv_train, pd.Series(results['train']['error'])], axis = 1)
    cv_valid = pd.concat([cv_valid, pd.Series(results['eval']['error'])], axis = 1)
    pd.to_pickle(cv_train, 'cv_results/actions_e20/train_err.p')
    pd.to_pickle(cv_valid, 'cv_results/actions_e20/validate_err.p')

In [ ]:
def cv_sessions(xgbInput):
    """Cross-validation on a subset of training data which meets a condition"""
    
    condition = 'dac_year == 2014 & action_counts != -1'
    idx = np.nonzero(xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index))
    idx = idx[0]
    np.random.shuffle(idx[0])
    n = len(idx)
    
    for i in range(0, kfolds):
        idx[0]
        
def cv_bydate(xgbInput):
    """Select folds for cross validation as all cases that occurred in a given with month 
    in 2014, with sessions data.
    Only cases in 2014 have sessions data and the last test case is on June 30.
    """
    
    for i in range(4,7):
        condition = 'dac_year == 2014 & dac_month == @i & action_counts != -1'
        valid_mask = xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index)
        valid_indx = np.where(valid_mask)[0]
        train_indx = np.where(~valid_mask)[0]
        yield train_indx, valid_indx

In [ ]:
train_data[train_data.index.isin(sessions[sessions.action_detail == 'change_contact_host_dates'].user_id)].country_destination.value_counts(normalize = True)

In [ ]:
null_tr_browsers = []
null_testbrowsers = []
for i in train_data.first_browser.unique():
    if i not in test_data.first_browser.unique():
        null_tr_browsers.append(i)
for i in test_data.first_browser.unique():
    if i not in train_data.first_browser.unique():
        null_testbrowsers.append(i)

In [ ]:
train_data[train_data.first_browser=='-unknown-'].first_browser.value_counts()

In [ ]:
train_data[train_data.affiliate_provider == "gsp"].country_destination.value_counts(normalize = True)

In [ ]:
print df.first_browser.value_counts()
print test_data.first_browser.value_counts()

In [ ]:
xgbInput.allDf.loc[xgbInput.allDf.query('1000 > age > 100').index, 'age'] = 105
#xgbInput.allDf.loc[self.allDf.query('age > 1000').index, 'age'] = 110

In [1]:
%run run_model.py

number of columns before one hot encoding 273
number of columns after one hot encoding 417


In [ ]:
np.where(av<14, -1)

In [ ]:
av = xgbInput.allDf.age.values

In [ ]:
with open('../xgbmodels/sessions_e20_25n.p') as f:
    bst = pickle.load(f)

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]